<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/pubmed_twadr_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'TwADR-L.fold-7.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-7.validation.csv",header=None)
valid.head()

,0,1
0,13,insomniac
1,24,racing heart
2,60,cigarette withdrawal
3,91,jitters
4,93,shooting


In [12]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-7.test.csv",header=None)
test.head()

,0,1
0,13,keeping me awake
1,94,anxiety
2,119,incr anxiety
3,119,anxiety
4,121,itching


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'TwADR_pubmed.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 'diclofenac',
 'was',
 'with',
 'were',
 'for',
 'pain',
 'sodium',
 'by',
 'drug',
 'on',
 'patients',
 'as',
 'that',
 'is',
 'at',
 'or',
 'mg',
 'increased',
 'group',
 'from',
 'mood',
 'this',
 'study',
 'treatment',
 'after',
 'p',
 'release',
 'an',
 'be',
 ',',
 'disorder',
 'me',
 'i',
 'abnormal',
 'withdrawal',
 'effect',
 'not',
 'decreased',
 'than',
 'compared',
 'weight',
 'anxiety',
 'using',
 'groups',
 'my',
 'are',
 'effects',
 'no',
 'two',
 'symptoms',
 'both',
 'inflammatory',
 'significantly',
 'between',
 'acute',
 'significant',
 'h',
 'potassium',
 'drugs',
 'anti',
 'skin',
 'day',
 'disorders',
 'voltaren',
 'syndrome',
 'blood',
 '(',
 ')',
 'time',
 'dose',
 'all',
 'behavior',
 'used',
 'these',
 'which',
 'sleeplessness',
 'it',
 'placebo',
 'results',
 '-',
 'had',
 'efficacy',
 'diseases',
 'days',
 'oral',
 'treated',


In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.844624,4.219815,0.346729,17:31


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.017951,3.831418,0.380907,17:21


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.738396,3.788031,0.385680,17:14


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.681271,3.754756,0.389706,17:25


In [25]:
learn.save_encoder('TwADR_pubmed_fold7_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('TwADR_pubmed_fold7_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.157570,6.459374,0.091549,00:23


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.180783,5.314089,0.140845,00:27


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.653076,4.923798,0.147887,01:08


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.365910,4.544039,0.169014,01:09
1,5.150282,4.377017,0.218310,01:09


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.887630,4.168142,0.260563,01:08
1,4.689466,4.115684,0.253521,01:05


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.505593,3.912461,0.274648,01:06
1,4.328784,3.824998,0.302817,01:10


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.131667,3.665275,0.330986,01:03
1,3.959701,3.610501,0.338028,01:07


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.831822,3.526082,0.302817,01:07
1,3.669475,3.435712,0.352113,01:10


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.493630,3.429317,0.380282,01:09
1,3.400767,3.474437,0.380282,01:09
2,3.227703,3.307821,0.422535,01:05
3,3.041924,3.299419,0.429577,01:08


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.885112,3.283533,0.408451,01:12
1,2.825374,3.287572,0.443662,01:07
2,2.663857,3.211102,0.450704,01:08
3,2.504497,3.170670,0.443662,01:09


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.314192,3.199917,0.450704,01:09
1,2.296424,3.194815,0.436620,01:08
2,2.157697,3.222903,0.464789,01:04
3,2.010880,3.214616,0.464789,01:02


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.832294,3.173965,0.471831,01:06
1,1.847323,3.244236,0.471831,01:11
2,1.703073,3.199125,0.450704,01:08
3,1.589845,3.188159,0.464789,01:07


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.460279,3.264130,0.457746,01:02
1,1.443911,3.376091,0.464789,01:07
2,1.333911,3.270062,0.457746,01:09
3,1.224264,3.267121,0.464789,01:04


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.150484,3.336891,0.450704,01:09
1,1.144051,3.438468,0.436620,01:07
2,1.044675,3.482291,0.471831,01:11
3,0.976385,3.476528,0.457746,01:07


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.900398,3.448976,0.471831,01:06
1,0.918291,3.577148,0.429577,01:06
2,0.859902,3.602414,0.443662,01:05
3,0.782764,3.620399,0.443662,01:06


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.724227,3.671462,0.464789,01:05
1,0.716937,3.602720,0.457746,01:06


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.691014,3.592876,0.464789,01:08
1,0.655891,3.685036,0.471831,01:08


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.591588,3.698958,0.457746,01:06


In [44]:
# save the best model

learn.save_encoder('TwADR_pubmed_fold7')

In [45]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.581096,3.775743,0.485915,01:07


In [46]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.609588,3.945838,0.436620,01:11


In [47]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.556426,3.811880,0.450704,01:04


In [51]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.576702,3.948550,0.443662,01:10


# Part three: Predict on the test dataset

In [52]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [53]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

142
142
[2191, 978, 978, 978, 121, 1096, 154, 1877, 2048, 189, 1340, 649, 231, 231, 1936, 2147, 2147, 233, 1936, 233, 452, 857, 300, 1054, 393, 1136, 418, 426, 518, 231, 1099, 749, 1340, 1748, 749, 1819, 749, 1597, 1932, 1932, 1798, 824, 1507, 1884, 852, 2149, 2103, 978, 974, 974, 1062, 224, 978, 978, 1729, 1130, 426, 1815, 2149, 1099, 1136, 1175, 1265, 852, 978, 1099, 121, 1308, 1484, 1327, 1718, 1507, 1483, 974, 1432, 1806, 418, 1701, 2149, 1573, 1594, 1597, 1597, 1597, 1597, 1597, 1597, 1597, 1597, 1597, 1610, 1610, 265, 2149, 1727, 1787, 1526, 1767, 1073, 1718, 1718, 1099, 1718, 1718, 1718, 561, 1597, 1787, 1099, 1727, 2191, 1787, 1597, 418, 2191, 109, 1373, 1915, 1936, 1936, 1936, 2149, 749, 1529, 1942, 1942, 2113, 16, 1265, 2048, 1813, 1295, 418, 1769, 2123, 2119, 2123, 2147, 1877, 2191, 1099, 1877]
[13, 94, 119, 119, 121, 143, 154, 189, 189, 189, 224, 230, 231, 231, 233, 233, 233, 233, 233, 233, 249, 265, 300, 359, 393, 396, 418, 426, 518, 596, 693, 712, 744, 749, 749, 749, 749,

In [54]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

51
0.3591549295774648
